# Importing The Necessary Libraries

In [ ]:
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import warnings

## Reading The Dataset

In [ ]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='yqrJt8DH_vOafkv6El54qhh7VLXgTsH9r303N1glb8ke',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'efficientwaterqualityanalysis-donotdelete-pr-kwte7fswwecp7x'
object_key = 'water_dataX.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

data = pd.read_csv(body)
data


## head() command is used to display the top n records of the specified dataset.

In [ ]:
data.head()

# Analysis Of The Data

## Descriptive Statistics Of The Dataset

In [ ]:
data.describe()

## info() method is used to get the columns in the dataset along with its properties

In [ ]:
data.info()

## The Above Description shows only the Descriptive Statistics of the year column beacuse it is the only column which has the datatype 'int64' 

In [ ]:
data.shape

## The Dataset contains 1991 Rows and 12 Columns.
## Exploratory Data Analysis
## Handling the Missing Values

In [ ]:
data.isnull().any()

In [ ]:
data.isnull().sum()

## Initially there is no Null Values present in the Dataset.


# Checking The Data Types Of The Columns


In [ ]:
data.dtypes

## Here many numeric columns also has the data type of object.

Let's convert the columns into float datatype.

In [ ]:
data['Temp']=pd.to_numeric(data['Temp'],errors='coerce')
data['D.O. (mg/l)']=pd.to_numeric(data['D.O. (mg/l)'],errors='coerce')
data['PH']=pd.to_numeric(data['PH'],errors='coerce')
data['B.O.D. (mg/l)']=pd.to_numeric(data['B.O.D. (mg/l)'],errors='coerce')
data['CONDUCTIVITY (µmhos/cm)']=pd.to_numeric(data['CONDUCTIVITY (µmhos/cm)'],errors='coerce')
data['NITRATENAN N+ NITRITENANN (mg/l)']=pd.to_numeric(data['NITRATENAN N+ NITRITENANN (mg/l)'],errors='coerce')
data['TOTAL COLIFORM (MPN/100ml)Mean']=pd.to_numeric(data['TOTAL COLIFORM (MPN/100ml)Mean'],errors='coerce')

In [ ]:
data.dtypes

## Let's Check For The Missing Values Again. 

In [ ]:
data.isnull().any()

In [ ]:
data.isnull().sum()

## Now some columns which have been converted to float datatype has some missing values.So, we have to replace the Null values using the mean or mode 

In [ ]:
data['Temp'].fillna(data['Temp'].mean(),inplace=True)
data['D.O. (mg/l)'].fillna(data['D.O. (mg/l)'].mean(),inplace=True)
data['PH'].fillna(data['PH'].mean(),inplace=True)
data['CONDUCTIVITY (µmhos/cm)'].fillna(data['CONDUCTIVITY (µmhos/cm)'].mean(),inplace=True)
data['B.O.D. (mg/l)'].fillna(data['B.O.D. (mg/l)'].mean(),inplace=True)
data['NITRATENAN N+ NITRITENANN (mg/l)'].fillna(data['NITRATENAN N+ NITRITENANN (mg/l)'].mean(),inplace=True)
data['TOTAL COLIFORM (MPN/100ml)Mean'].fillna(data['TOTAL COLIFORM (MPN/100ml)Mean'].mean(),inplace=True)


In [ ]:
data.isnull().any()

In [ ]:
data.isnull().sum()

## Now, There is no missing values present in any columns of the dataset.So, we have handled the Missing Values present in the dataset. 

In [ ]:
data.head()

In [ ]:
data.drop(["FECAL COLIFORM (MPN/100ml)"],axis=1,inplace=True)

In [ ]:
data.head()

## Renaming The Column Names in the Dataset For simplicity 


In [ ]:
data=data.rename(columns = {'D.O. (mg/l)':'do'})
data=data.rename(columns = {'CONDUCTIVITY (µmhos/cm)':'co'})
data=data.rename(columns = {'B.O.D. (mg/l)':'bod'})
data=data.rename(columns = {'NITRATENAN N+ NITRITENANN (mg/l)':'na'})
data=data.rename(columns = {'TOTAL COLIFORM (MPN/100ml)Mean':'tc'})
data=data.rename(columns = {'STATION CODE':'station'})
data=data.rename(columns = {'LOCATIONS':'location'})
data=data.rename(columns = {'STATE':'state'})
data=data.rename(columns = {'PH':'ph'})

In [ ]:
data.head()

## So, The Dataset has been read , analysed and preprocessing has been done successfully.


# Water Quality Index Calculation

## Calculation of pH (in range of 1 to 100)

In [ ]:

data['nph']=data.ph.apply(lambda x: (100 if (8.5>=x>=7)  
                                 else(80 if  (8.6>=x>=8.5) or (6.9>=x>=6.8) 
                                      else(60 if (8.8>=x>=8.6) or (6.8>=x>=6.7) 
                                          else(40 if (9>=x>=8.8) or (6.7>=x>=6.5)
                                              else 0)))))

## Calculation of Dissolved Oxygen(in range of 1 to 100)

In [ ]:

data['ndo']=data.do.apply(lambda x: (100 if (x>=6)
                                 else(80 if(6>=x>=5.1)
                                      else(60 if (5>=x>=4.1)
                                           else(40 if (4>=x>=3)
                                                else 0)))))

## Calculation of Total Coliform(in range of 1 to 100) 

In [ ]:

data['nco']=data.tc.apply(lambda x: (100 if (5>=x>=0)
                                 else(80 if(50>=x>=5)
                                      else(60 if (500>=x>=50)
                                           else(40 if (10000>=x>=500)
                                                else 0)))))

## Calculation of B.O.D - Biological Oxygen Demand (in range of 1 to 100) 

In [ ]:

data['nbdo']=data.bod.apply(lambda x: (100 if (3>=x>=0)
                                 else(80 if(6>=x>=3)
                                      else(60 if (80>=x>=6)
                                           else(40 if (125>=x>=80)
                                                else 0)))))

##  Calculation of Electrical Conductivity (in range of 1 to 100)

In [ ]:

data['nec']=data.co.apply(lambda x: (100 if (75>=x>=0)
                                 else(80 if(150>=x>=75)
                                      else(60 if (225>=x>=150)
                                           else(40 if (300>=x>=225)
                                                else 0)))))

## Calculation of Nitrate Content(in range of 1 to 100) 

In [ ]:


#calculation of nitrate
data['nna']=data.na.apply(lambda x: (100 if (20>=x>=0)
                                 else(80 if(50>=x>=20)
                                      else(60 if (100>=x>=50)
                                           else(40 if (200>=x>=100)
                                                else 0)))))

In [ ]:
data.head()

In [ ]:
data.dtypes

# Calculating Water Quality Index Value


In [ ]:

data['wph']=data.nph * 0.165
data['wdo']=data.ndo * 0.281
data['wbdo']=data.nbdo * 0.234
data['wec']=data.nec* 0.009
data['wna']=data.nna * 0.028
data['wco']=data.nco * 0.281
data['wqi']=data.wph+data.wdo+data.wbdo+data.wec+data.wna+data.wco 

In [ ]:
data

In [ ]:
data.head()

## Calculating Overall Water Quality Index for Each Year

In [ ]:
average=data.groupby('year')['wqi'].mean()

In [ ]:
average

In [ ]:
data1=average.reset_index(level=0,inplace=False)
data1

# Various Data Visualisations

## 1. Univariate Visualisations

In [ ]:
for col in data.columns:
    if data.dtypes[col]=='float64'or data.dtypes[col]=='int64':
        sns.boxplot(x=data[col]).set(xlabel=col)
        plt.show()

In [ ]:
for col in data.columns:
    if data.dtypes[col]=='float64':
        sns.distplot(x=data[col]).set(xlabel=col)
        plt.show()

## 2. Bivariate Analysis 

In [ ]:
 for col1 in data.columns:
    if data.dtypes[col1]=='float64':
        for col2 in ['wqi']:
            if data.dtypes[col2]=='float64':
                sns.boxplot(x=data[col1],y=data[col2]).set(xlabel=[col1],ylabel=[col2])
                plt.show()

## 3. Multivariate Analysis

In [ ]:
data.corr()

In [ ]:
 sns.heatmap(data.corr(),annot=True)

## Removing Unnecessary Columns in the Dataset which is used to calculate the Water Quality Index (WQI)

In [ ]:
data.drop(['Temp','station','location','state','nbdo',"nec","nna","wph","wdo","wbdo","wec","wna","wco","nph","ndo","nco"],axis = 1,inplace=True)
data.head()

In [ ]:
data.corr()

In [ ]:
sns.heatmap(data.corr(),annot=True)

## Data has been visualization successfully

# Splitting Dataset into Dependent and Independent Columns

In [ ]:
x=data.iloc[:,0:7].values
y=data.iloc[:,7:].values

In [ ]:
x.shape

In [ ]:
y.shape

In [ ]:
x

In [ ]:
y

# Splitting The Data Into Training And Testing

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state=10)


In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

##  We have calculated the Water Quality Index value from various WQI determining parameters and performed various visualisations in the Dataset.

##  Also, we have split the Dependent and Independent columns and also have split the data into training and testing. 

# Building Random Forest Regression Model

In [ ]:

#Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
regressor.fit(x_train, y_train)
y_pred = regressor.predict(x_test)

# Model Evaluation

In [ ]:
from sklearn import metrics
print('MAE:',metrics.mean_absolute_error(y_test,y_pred))
print('MSE:',metrics.mean_squared_error(y_test,y_pred))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test,y_pred)))


In [ ]:
#accuracy of the model
metrics.r2_score(y_test, y_pred)

# Saving The Model

In [ ]:
import pickle
pickle.dump(regressor,open('wqi.pkl', 'wb'))
model = pickle.load(open('wqi.pkl','rb'))

In [ ]:
!pip install -U ibm-watson-machine-learning

In [ ]:
from ibm_watson_machine_learning import APIClient

credentials = {
                    "url":"https://us-south.ml.cloud.ibm.com",
                    "apikey":"Ivs8Ix9Pkwz2gnW9alUtCeSQkrNtxT8ZdeJHd9ZuAL_o"
                  }
client = APIClient(wml_credentials)

In [ ]:
def guid_from_space_name(client,space_name):
    space = client.spaces.get_details()
#         print(space)
    return(next(item for item in space['resources'] if item['entity']['name']==space_name)['metadata']['id']) 

In [ ]:
space_uid = guid_from_space_name(client,'Models')
print("Spcae UID = "+ space_uid)

In [ ]:
client.set.default_space(space_uid)

In [ ]:
client.software_specifications.list()

In [ ]:
software_spec_uid = client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")
#software_spec_uid = client.software_specifications.get_uid_by_name("default_py3.6")
software_spec_uid

In [ ]:
model_details = client.repository.store_model(model=regressor,meta_props={
    client.repository.ModelMetaNames.NAME:"water_dataX",
    client.repository.ModelMetaNames.TYPE:"scikit-learn_1.0	",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid}
                                             )
model_id=client.repository.get_model_id(model_details)

In [ ]:
model_id

In [ ]:
x_train[0]

In [ ]:
regressor.predict([[-0.684425  , -0.06140958, -0.20384397,  0.06998719,  0.86213052,
       -0.042079]])

In [ ]:
model_id = client.repository.get_model_id(model_details)
model_id

In [ ]:
x_train[0]

In [ ]:
water.predict([5.500e+00, 7.400e+00, 6.350e+02, 8.800e+00, 5.080e+00, 5.500e+03,2.012e+03])